In [2]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

# Customers who reported to be Defrauded or Hacked - get this list from the Ops team

In [2]:
dfhacked = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\Demo&Telco Scorecard\Data\SupportingFile\Database of internal & external fraud incidents v2.xlsx", sheet_name = 'Main')

c:\Users\DwaipayanChakroborti\Myenv\reporting\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [3]:
dfhacked.columns

Index(['Source', 'Ticket Number', 'Ticket Status', 'Report Date', 'Fraud Type',
       'Sub MO', 'Disputed Amount', 'OFI', 'RFI', 'Complainant Name',
       'Contact Information', 'Source Account Number', 'With Police Report?',
       'TSA/Account Number', 'PID', 'Account Name', 'Segment/KYC', 'Address1',
       'Address2', 'Address3', 'Mobile Number', 'Account Status',
       'Date Blocked', 'TDB Account Balance', 'With TD?', 'With Active Loans?',
       'Callout Result', 'Disposition', 'Actions'],
      dtype='object')

In [4]:
pd.set_option("Display.max_columns", None)
dfhacked.head()

,Source,Ticket Number,Ticket Status,Report Date,Fraud Type,Sub MO,Disputed Amount,OFI,RFI,Complainant Name,Contact Information,Source Account Number,With Police Report?,TSA/Account Number,PID,Account Name,Segment/KYC,Address1,Address2,Address3,Mobile Number,Account Status,Date Blocked,TDB Account Balance,With TD?,With Active Loans?,Callout Result,Disposition,Actions
0,JIRA,CC-1686423,Closed,2023-05-22,Internal Fraud,Advance Fee Scam,1500.0,Various,CIMB via DragonPay,KATHLEEN ROSE BAUTISTA ANDAL,NaN,Various,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,Valid,"Joint investigation of InfoSec, AF Ops and HR ..."
1,NaN,CC-1698906,Closed,2023-05-26,NaN,NaN,1500.0,NaN,NaN,JESSA NA NAZ,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,CC-1698935,Closed,2023-05-26,NaN,NaN,2000.0,NaN,NaN,ROLLEN CARILLO REGINO,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,CC-1698352,Closed,2023-05-26,NaN,NaN,1500.0,NaN,NaN,JAIME FELIX REY III GENERAO CRUZ,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,CC-1698914,Closed,2023-05-26,NaN,NaN,1500.0,NaN,NaN,MAYDEN CORPUZ MAGLACION,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
dfhacked['Disposition'].value_counts()

Disposition
Not Valid    162
Valid          2
Name: count, dtype: int64

In [6]:
dfhacked = dfhacked[dfhacked['Disposition']=='Valid']
dfhacked.shape

(2, 29)

In [7]:
# Replace spaces with underscores in column names
dfhacked.columns = dfhacked.columns.str.replace(' ', '_')
dfhacked.columns = dfhacked.columns.str.replace('/', '_')
dfhacked.columns = dfhacked.columns.str.replace('?', '')

In [8]:
dfhacked.dtypes

Source                           object
Ticket_Number                    object
Ticket_Status                    object
Report_Date              datetime64[ns]
Fraud_Type                       object
Sub_MO                           object
Disputed_Amount                 float64
OFI                              object
RFI                              object
Complainant_Name                 object
Contact_Information              object
Source_Account_Number            object
With_Police_Report               object
TSA_Account_Number              float64
PID                             float64
Account_Name                     object
Segment_KYC                      object
Address1                         object
Address2                         object
Address3                         object
Mobile_Number                   float64
Account_Status                   object
Date_Blocked             datetime64[ns]
TDB_Account_Balance              object
With_TD                          object


In [9]:
dfhacked1 = dfhacked[['Source', 'Ticket_Number', 'Ticket_Status', 'Report_Date', 'Fraud_Type',
       'Sub_MO', 'Disputed_Amount', 'OFI', 'RFI', 'Complainant_Name',
       'Contact_Information', 'With_Police_Report',
       'TSA_Account_Number', 'PID', 'Account_Name', 'Segment_KYC', 'Address1',
       'Address2', 'Address3', 'Mobile_Number', 'Account_Status',
       'Date_Blocked', 'With_TD', 'With_Active_Loans',
       'Callout_Result', 'Disposition', 'Actions']].copy()

In [10]:
from google.cloud import bigquery

# Define the dataset and table references
dataset_id = 'dap_ds_poweruser_playground'
table_id = 'Defraudedandhacked_mu'

# Define the table schema as per your DataFrame columns
schema = [
    bigquery.SchemaField("Source", "STRING"),
    bigquery.SchemaField("Ticket_Number", "STRING"),
    bigquery.SchemaField("Ticket_Status", "STRING"),
    bigquery.SchemaField("Report_Date", "TIMESTAMP"),
    bigquery.SchemaField("Fraud_Type", "STRING"),
    bigquery.SchemaField("Sub_MO", "STRING"),
    bigquery.SchemaField("Disputed_Amount", "FLOAT"),
    bigquery.SchemaField("OFI", "STRING"),
    bigquery.SchemaField("RFI", "STRING"),
    bigquery.SchemaField("Complainant_Name", "STRING"),
    bigquery.SchemaField("Contact_Information", "STRING"),
    bigquery.SchemaField("With_Police_Report", "STRING"),
    bigquery.SchemaField("TSA_Account_Number", "FLOAT"),
    bigquery.SchemaField("PID", "FLOAT"),
    bigquery.SchemaField("Account_Name", "STRING"),
    bigquery.SchemaField("Segment_KYC", "STRING"),
    bigquery.SchemaField("Address1", "STRING"),
    bigquery.SchemaField("Address2", "STRING"),
    bigquery.SchemaField("Address3", "STRING"),
    bigquery.SchemaField("Mobile_Number", "FLOAT"),
    bigquery.SchemaField("Account_Status", "STRING"),
    bigquery.SchemaField("Date_Blocked", "TIMESTAMP"),
    bigquery.SchemaField("With_TD", "STRING"),
    bigquery.SchemaField("With_Active_Loans", "STRING"),
    bigquery.SchemaField("Callout_Result", "STRING"),
    bigquery.SchemaField("Disposition", "STRING"),
    bigquery.SchemaField("Actions", "STRING"),
]

# Create the dataset reference
dataset_ref = client.dataset(dataset_id)

# Define the table reference
table_ref = dataset_ref.table(table_id)

# Configure the job to overwrite the table if it already exists
job_config = bigquery.LoadJobConfig(schema=schema)
job_config.autodetect = False
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

# Convert datetime columns to timestamp
for col in dfhacked.select_dtypes(include=['datetime64']):
    dfhacked[col] = dfhacked[col].astype('int64') // 10**9

# Load the DataFrame into BigQuery
job = client.load_table_from_dataframe(dfhacked1, table_ref, job_config=job_config)

# Wait for the job to complete
job.result()

print(f"Table {table_id} created in dataset {dataset_id}.")

Table Defraudedandhacked_mu created in dataset dap_ds_poweruser_playground.


In [11]:
sq = """select * from dap_ds_poweruser_playground.Defraudedandhacked_mu;"""
checkdf = client.query(sq).to_dataframe()
checkdf.shape

(2, 27)

# Query for Aquisition Channel

In [12]:
sq = """WITH installs AS 
(
SELECT
Event_Time,
CASE 
    ### social media ###
    -- anything from social
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%social%' THEN 'Social'
    ### direct ###
    -- fwb / referral
    WHEN COALESCE(Media_Source,Partner,Campaign) = 'af_app_invites' THEN 'FWB'
    WHEN (Media_Source = 'invalid_media_source_name' OR Media_Source IS NULL) AND LOWER(Campaign) LIKE '%refer%' THEN 'FWB'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%refer%' THEN 'FWB'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%moengage%' THEN 'FWB'
    -- apple search
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%apple search ads%' THEN 'AppleSearch'
    -- facebook / meta
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%Facebook Ads%' THEN 'Ads_Meta'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%meta%' THEN 'Ads_Meta'
    WHEN COALESCE(Media_Source,Partner,Campaign) = 'facebook' THEN 'Ads_Meta'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) = 'restricted' THEN 'Ads_Meta'
    -- google
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%googleadwords%' THEN 'Ads_Google'
      -- tiktok
    WHEN COALESCE(Media_Source,Partner,Campaign) like '%bytedanceglobal%' THEN 'TikTok_Paid'
    ### affiliates ###
    -- pokkt source
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%pokkt%' THEN 'Aff_Pokkt'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%Tonik_CPA%' THEN 'Aff_Pokkt'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%campaign_name%' THEN 'Aff_Pokkt'
    -- tyr ads
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) IN ('tyrads_int','tjzymob_int','tyrads','ta_tonik_aos_ph') THEN 'Aff_TyrAds'
    -- sales doubler
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%salesdoubler%' THEN 'Aff_SalesDoubler'
     
    -- imoney
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%imoney%' THEN 'Aff_iMoney'
    -- moneymax
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%moneymax%' THEN 'Aff_MoneyMax'
    -- jeff
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%jeff%' THEN 'Aff_Jeff'
    -- shareit
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%shareit%' THEN 'Aff_ShareIt'
     
    -- appnext
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%appnext%' THEN 'Aff_AppNext'
    -- mediadonuts / entravision
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%mediadonuts%' THEN 'Aff_Mediadonuts'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%entravision%' THEN 'Aff_Mediadonuts'
    -- shopback
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%shopback_int%' THEN 'Aff_Shopback'
    -- deepsea
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%deepsea%' THEN 'Aff_DeepSea'
    ## organic ##
    WHEN COALESCE(Media_Source,Partner,Campaign) IN ('af_banner') THEN 'af_Banner'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%sendgrid%' THEN 'Organic'
    WHEN COALESCE(Media_Source,Partner,Campaign) IN ('Website') THEN 'Website'
    
    ## owned media ##
    WHEN media_source IN ('PRFlexLoans_WebAndroidPressRelease') THEN 'Owned Media'
    
    WHEN media_source IN ('invalid_media_source_name') THEN 'Invalid Media Source'
    WHEN media_source IN ('None') THEN 'Broken OneLink'
ELSE "Organic"
END as Source,
AppsFlyer_ID,
Customer_User_ID
    
FROM
(
    SELECT Install_Time,Event_Time,AppsFlyer_ID,Customer_User_ID,Platform,Partner,Media_Source,Campaign,Device_Type,OS_Version
    FROM `prj-prod-dataplatform.appsflyer_raw.organic_installs_report` I --Organic Installs
    WHERE Date(Event_Time) BETWEEN DATE_SUB(CURRENT_DATE, INTERVAL 11 MONTH) AND DATE_SUB(CURRENT_DATE, INTERVAL 1 DAY)
    AND DATE(_PARTITIONTIME) BETWEEN DATE_SUB(CURRENT_DATE, INTERVAL 350 DAY) AND DATE_SUB(CURRENT_DATE, INTERVAL 1 DAY)
    UNION ALL
    SELECT Install_Time,Event_Time,AppsFlyer_ID,Customer_User_ID,Platform,Partner,Media_Source,Campaign,Device_Type,OS_Version
    FROM `prj-prod-dataplatform.appsflyer_raw.installs_report` I -- Non-Organic Installs
    WHERE Date(Event_Time) BETWEEN DATE_SUB(CURRENT_DATE, INTERVAL 11 MONTH) AND DATE_SUB(CURRENT_DATE, INTERVAL 1 DAY)
    AND DATE(_PARTITIONTIME) BETWEEN DATE_SUB(CURRENT_DATE, INTERVAL 350 DAY) AND DATE_SUB(CURRENT_DATE, INTERVAL 1 DAY)
    
)
WHERE 1=1
AND event_time >= '2022-09-01'
)
, registrations AS 
(
SELECT
MAX(Event_Time) event_time,
CASE 
    ### social media ###
    -- anything from social
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%social%' THEN 'Social'
    ### direct ###
    -- fwb / referral
    WHEN COALESCE(Media_Source,Partner,Campaign) = 'af_app_invites' THEN 'FWB'
    WHEN (Media_Source = 'invalid_media_source_name' OR Media_Source IS NULL) AND LOWER(Campaign) LIKE '%refer%' THEN 'FWB'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%refer%' THEN 'FWB'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%moengage%' THEN 'FWB'
    -- apple search
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%apple search ads%' THEN 'AppleSearch'
    -- facebook / meta
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%Facebook Ads%' THEN 'Ads_Meta'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%meta%' THEN 'Ads_Meta'
    WHEN COALESCE(Media_Source,Partner,Campaign) = 'facebook' THEN 'Ads_Meta'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) = 'restricted' THEN 'Ads_Meta'
    -- google
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%googleadwords%' THEN 'Ads_Google'
      -- tiktok
    WHEN COALESCE(Media_Source,Partner,Campaign) like '%bytedanceglobal%' THEN 'TikTok_Paid'
    ### affiliates ###
    -- pokkt source
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%pokkt%' THEN 'Aff_Pokkt'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%Tonik_CPA%' THEN 'Aff_Pokkt'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%campaign_name%' THEN 'Aff_Pokkt'
    -- tyr ads
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) IN ('tyrads_int','tjzymob_int','tyrads','ta_tonik_aos_ph') THEN 'Aff_TyrAds'
    -- sales doubler
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%salesdoubler%' THEN 'Aff_SalesDoubler'
     
    -- imoney
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%imoney%' THEN 'Aff_iMoney'
    -- moneymax
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%moneymax%' THEN 'Aff_MoneyMax'
    -- jeff
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%jeff%' THEN 'Aff_Jeff'
    -- shareit
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%shareit%' THEN 'Aff_ShareIt'
     
    -- appnext
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%appnext%' THEN 'Aff_AppNext'
    -- mediadonuts / entravision
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%mediadonuts%' THEN 'Aff_Mediadonuts'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%entravision%' THEN 'Aff_Mediadonuts'
    -- shopback
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%shopback_int%' THEN 'Aff_Shopback'
    -- deepsea
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%deepsea%' THEN 'Aff_DeepSea'
    ## organic ##
    WHEN COALESCE(Media_Source,Partner,Campaign) IN ('af_banner') THEN 'af_Banner'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%sendgrid%' THEN 'Organic'
    WHEN COALESCE(Media_Source,Partner,Campaign) IN ('Website') THEN 'Website'
    
    ## owned media ##
    WHEN media_source IN ('PRFlexLoans_WebAndroidPressRelease') THEN 'Owned Media'
    
    WHEN media_source IN ('invalid_media_source_name') THEN 'Invalid Media Source'
    WHEN media_source IN ('None') THEN 'Broken OneLink'
ELSE "Organic"
END as Source,
AppsFlyer_ID,
-- Customer_User_ID
FROM
(

  SELECT MIN(Event_time) event_time,Media_source,Campaign,Partner,AppsFlyer_ID,Install_Time,Platform,OS_Version,Device_Type
  FROM `prj-prod-dataplatform.appsflyer_raw.in_app_events_report`  -- Non-Organic Registrations
  WHERE (Event_Name='Onboarding_Success' OR Event_Name LIKE 'Onboarding_Success_Dashboard%')
  AND DATE(_PARTITIONTIME) BETWEEN DATE_SUB(CURRENT_DATE, INTERVAL 350 DAY) AND DATE_SUB(CURRENT_DATE, INTERVAL 1 DAY)
  AND DATE(Event_Time) BETWEEN DATE_SUB(CURRENT_DATE, INTERVAL 11 MONTH) AND DATE_SUB(CURRENT_DATE, INTERVAL 1 DAY)
  GROUP BY 2,3,4,5,6,7,8,9
  UNION ALL
      
  SELECT MIN(Event_time) event_time,Media_source,Campaign,Partner,AppsFlyer_ID,Install_Time,Platform,OS_Version,Device_Type
  FROM `prj-prod-dataplatform.appsflyer_raw.organic_in_app_events_report`  -- Organic Registrations
  WHERE (Event_Name='Onboarding_Success' OR Event_Name LIKE 'Onboarding_Success_Dashboard%')
  AND DATE(_PARTITIONTIME) BETWEEN DATE_SUB(CURRENT_DATE, INTERVAL 380 DAY) AND DATE_SUB(CURRENT_DATE, INTERVAL 1 DAY)
  AND DATE(Event_Time) BETWEEN DATE_SUB(CURRENT_DATE, INTERVAL 11 MONTH) AND DATE_SUB(CURRENT_DATE, INTERVAL 1 DAY)
  GROUP BY 2,3,4,5,6,7,8,9
    
)
WHERE 1=1
AND event_time >= '2022-09-01'
GROUP BY 2,3
)
, applications AS 
(
SELECT 
decision_date,
IF(reloan_flag=1,'Re-loan',new_loan_type) loan_type,
CASE 
    ### social media ###
    -- anything from social
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%social%' THEN 'Social'
    ### direct ###
    -- fwb / referral
    WHEN COALESCE(Media_Source,Partner,Campaign) = 'af_app_invites' THEN 'FWB'
    WHEN (Media_Source = 'invalid_media_source_name' OR Media_Source IS NULL) AND LOWER(Campaign) LIKE '%refer%' THEN 'FWB'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%refer%' THEN 'FWB'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%moengage%' THEN 'FWB'
    -- apple search
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%apple search ads%' THEN 'AppleSearch'
    -- facebook / meta
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%Facebook Ads%' THEN 'Ads_Meta'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%meta%' THEN 'Ads_Meta'
    WHEN COALESCE(Media_Source,Partner,Campaign) = 'facebook' THEN 'Ads_Meta'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) = 'restricted' THEN 'Ads_Meta'
    -- google
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%googleadwords%' THEN 'Ads_Google'
      -- tiktok
    WHEN COALESCE(Media_Source,Partner,Campaign) like '%bytedanceglobal%' THEN 'TikTok_Paid'
    ### affiliates ###
    -- pokkt source
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%pokkt%' THEN 'Aff_Pokkt'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%Tonik_CPA%' THEN 'Aff_Pokkt'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%campaign_name%' THEN 'Aff_Pokkt'
    -- tyr ads
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) IN ('tyrads_int','tjzymob_int','tyrads','ta_tonik_aos_ph') THEN 'Aff_TyrAds'
    -- sales doubler
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%salesdoubler%' THEN 'Aff_SalesDoubler'
     
    -- imoney
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%imoney%' THEN 'Aff_iMoney'
    -- moneymax
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%moneymax%' THEN 'Aff_MoneyMax'
    -- jeff
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%jeff%' THEN 'Aff_Jeff'
    -- shareit
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%shareit%' THEN 'Aff_ShareIt'
     
    -- appnext
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%appnext%' THEN 'Aff_AppNext'
    -- mediadonuts / entravision
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%mediadonuts%' THEN 'Aff_Mediadonuts'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%entravision%' THEN 'Aff_Mediadonuts'
    -- shopback
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%shopback_int%' THEN 'Aff_Shopback'
    -- deepsea
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%deepsea%' THEN 'Aff_DeepSea'
    ## organic ##
    WHEN COALESCE(Media_Source,Partner,Campaign) IN ('af_banner') THEN 'af_Banner'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%sendgrid%' THEN 'Organic'
    WHEN COALESCE(Media_Source,Partner,Campaign) IN ('Website') THEN 'Website'
    
    ## owned media ##
    WHEN media_source IN ('PRFlexLoans_WebAndroidPressRelease') THEN 'Owned Media'
    
    WHEN media_source IN ('invalid_media_source_name') THEN 'Invalid Media Source'
    WHEN media_source IN ('None') THEN 'Broken OneLink'
ELSE "Organic"
END as Source,
    
digitalLoanAccountId,
flagApproval,
customerId,
Appsflyer_id
FROM `prj-prod-dataplatform.risk_credit_mis.loan_master_table` LA
WHERE 1=1 
AND DATE(LA.decision_date) >= '2023-07-01'
AND new_loan_Type IN ('Quick','Flex')
ORDER BY 1 
)
select customerId, Appsflyer_id, digitalLoanAccountId,Source  
, case when Source like 'Organic' then 'Organic' else 'InOrganic' end flag_organic_inorganic
from applications;"""

In [13]:
dfaquisition = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfaquisition.shape

Job ID ca701ad5-951e-4dd8-8ded-d622efd431c1 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


(443827, 5)

In [27]:
dfaquisition.columns

Index(['customerId', 'Appsflyer_id', 'digitalLoanAccountId', 'Source',
       'flag_organic_inorganic'],
      dtype='object')

# Query for the rest of the part

In [14]:
sq = """
with base as 
(select * from
(select  distinct customerId,user_id,tsaOnboardingDateTime,credolab_ReferenceNumber,deviceType,
digitalLoanAccountId,case when loan_type in ('Flex-up','Reloan') then 'Repeat Applicant' 
when First_application=1 and customer_id_first_applicant is not null then 'New Applicant'
when accept_cust_id is null then  'Prev_Rejected_Reapply'
when rn=1 then 'New Applicant' 
--when Reject_application_count=1 then 'New Applicant' 
when accept_application_count>1 and accept_cust_id is not null then  'Repeat Applicant'

else 'Repeat Applicant' 
end as applicant_type,loan_type,loan_sub_type,sil_product,application_Submission_DateTime,
fraudScore,credolabScore,decision,applicationStatus,disbursementFlag,flg_mature_fpd10,flg_mature_fpd30,
flg_mature_fspd_30,flg_mature_fstpd_30,deffpd10,deffpd30,deffspd30,deffstpd30

from (with deliquency as
(select loanAccountNumber,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30  in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),

accept_application_cnt as (select customerId,count(*) as accept_application_count
from risk_credit_mis.loan_master_table loan where loan.new_loan_type in ('Flex-up','SIL-Instore','SIL-Online(discontinued)','Quick','SIL ZERO','ACL Patner 711')
  and (flagApproval = 1 or flagRejection = 1 or loan.new_loan_type in ('Flex-up')) and coalesce(loan.termsAndConditionsSubmitDateTime,if (loan.new_loan_type ='Flex-up',loan.startApplyDateTime,loan.termsAndConditionsSubmitDateTime))  is not null and applicationStatus in ('ACTIVATED','COMPLETED','APPROVED',
'INPROGRESS','SIGNEDDOCCOMPLETED','ACCEPT') group by 1 order by customerId),


applciant_type_first as (select customerId customer_id_first_applicant,count(*) First_application from `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 
where new_loan_type in ('Flex-up','SIL-Instore','SIL-Online(discontinued)','Quick','SIL ZERO','ACL Patner 711')
  and (flagApproval = 1 or flagRejection = 1 or new_loan_type in ('Flex-up')) and coalesce(termsAndConditionsSubmitDateTime,if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime))  is not null group by 1 having count(*)=1),

previous_loan_product as (select loan.customerId,
    case when loan.new_loan_type ='Flex-up' and reloan_flag=1 then 'Reloan'
    when loan.new_loan_type ='Flex-up' and reloan_flag=0 then 'Flex-up'
     WHEN loan.new_loan_type ='Flex' then 'Flex'
    when loan.new_loan_type ='Quick'  then 'Quick'
    when loan.new_loan_type ='Flex-up' and reloan_flag=0 then 'Flex-up'
    when loan.new_loan_type in ('SIL-Instore','SIL-Online(discontinued)','SIL ZERO') then 'SIL'
    when loan.new_loan_type in ('ACL Patner 711') then '711'
    end as prev_new_loan_type,loan.loanType prev_loan_type,prev_loan.previous_loan_acct,prev_loan.loanAccountNumber,loan.startApplyDateTime from `risk_credit_mis.loan_master_table` loan
join
(select * from (select customerId, digitalLoanAccountId,loanAccountNumber,new_loan_type,startApplyDateTime, lead(loanAccountNumber)
 OVER(PARTITION BY customerId order by startApplyDateTime desc) as previous_loan_acct from `risk_credit_mis.loan_master_table`
 where applicationStatus in ('COMPLETED','ACTIVATED')  order by customerId) where --previous_loan_acct is not null and 
 new_loan_type='Flex-up')prev_loan on loan.loanAccountNumber=prev_loan.previous_loan_acct)
  
  select
  loan.customerId,
  cust.user_id,
  tsa_onboarding_time as tsaOnboardingDateTime,
  credolabRefNumber as credolab_ReferenceNumber,
  deviceType,
  loan.digitalLoanAccountId,
  --case when applciant_type_first.disbursed_loan >1 then 'Repeat Applicant' else 'New Applicant' end as applicant_type,

    case when loan.new_loan_type ='Flex-up' and reloan_flag=1 then 'Reloan'
    when loan.new_loan_type ='Flex-up' and reloan_flag=0 then 'Flex-up'
    when loan.new_loan_type ='Quick'  then 'Quick'
    when loan.new_loan_type ='Flex-up' and reloan_flag=0 then 'Flex-up'
    when loan.new_loan_type in ('SIL-Instore','SIL-Online(discontinued)','SIL ZERO') then 'SIL'
    when loan.new_loan_type in ('ACL Patner 711') then '711'
    end as loan_type,
    case when loan.new_loan_type ='Flex-up' and reloan_flag=1 then coalesce('Reloan_'||prev_new_loan_type)
    when loan.new_loan_type ='Flex-up' and reloan_flag=0 then coalesce('Flex-up_'||prev_new_loan_type)
    else loan.new_loan_type end loan_sub_type,
    
    --new_loan_type as sub_loan_type,reloan_flag,
    --prev_new_loan_type, 
    case when loan.loantype='BNPL' and store_type =1 then 'Appliance'
    when loan.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loan.loantype='BNPL' and store_type =3 then 'Mall' 
    when loan.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as sil_product,
    coalesce(loan.termsAndConditionsSubmitDateTime,if (loan.new_loan_type ='Flex-up',loan.startApplyDateTime,loan.termsAndConditionsSubmitDateTime)) application_Submission_DateTime,
    fraudScore,credolabScore,
      case
      when flagApproval = 1 then 'approved'
      when flagRejection = 1 then 'rejected'
      else 'auto-approved without model'
    end as decision,
    applicationStatus,
    flagDisbursement as disbursementFlag,
    IF(Date_diff(current_date(),date(disbursementDateTime), DAY) >= 40, 1 ,0) as flg_mature_fpd10,
    IF(Date_diff(current_date(),date(disbursementDateTime), DAY) >= 60, 1 ,0) as flg_mature_fpd30,
    IF(Date_diff(current_date(),date(disbursementDateTime), DAY) >= 90, 1 ,0) as flg_mature_fspd_30,
    IF(Date_diff(current_date(),date(disbursementDateTime), DAY) >= 120, 1 ,0) as flg_mature_fstpd_30,
    deliquency.deffpd10,deliquency.deffpd30,deliquency.deffspd30,deliquency.deffstpd30,row_number() over (partition by loan.customerId order by coalesce(termsAndConditionsSubmitDateTime,if (new_loan_type ='Flex-up',loan.startApplyDateTime,termsAndConditionsSubmitDateTime))) rn,accept_application_count,accept_application_cnt.customerId accept_cust_id,First_application,customer_id_first_applicant

   
  from prj-prod-dataplatform.risk_credit_mis.loan_master_table loan
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name) sil_category on purpleKey=sil_category.mer_refferal_code
  left join deliquency on deliquency.loanAccountNumber=loan.loanAccountNumber
  left join previous_loan_product on previous_loan_product.loanAccountNumber=loan.loanAccountNumber
  left join applciant_type_first on applciant_type_first.customer_id_first_applicant=loan.customerId
  left join dl_customers_db_raw.tdbk_customer_mtb cust on cust.cust_id=cast(loan.customerId as string)
  left join accept_application_cnt on accept_application_cnt.customerId=loan.customerId
  where loan.new_loan_type in ('Flex-up','SIL-Instore','SIL-Online(discontinued)','Quick','SIL ZERO','ACL Patner 711')
  and (flagApproval = 1 or flagRejection = 1 or loan.new_loan_type in ('Flex-up')) and coalesce(loan.termsAndConditionsSubmitDateTime,if (loan.new_loan_type ='Flex-up',loan.startApplyDateTime,loan.termsAndConditionsSubmitDateTime))  is not null) final where loan_sub_type is not null)
), 
delqtab as 
(select loanAccountNumber,  
case when obs_min_inst_def30 >=2 then 1 else 0 end obsFSPD30,
case when min_inst_def30 in (1,2) then 1 else 0 end fspd30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
Excl_evertransacted as 
(select distinct cast(customer_id as numeric) customerId from prj-prod-dataplatform.risk_mart.customer_transactions where status like 'Success'),
Excl_qualified_FSPD30 as
(select loanAccountNumber, obs_min_inst_def30 from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data where obs_min_inst_def30 >= 2),
Excl_blacklisted_clean as 
(select customerid, matching_attribute, id, refresh_date from  prj-prod-dataplatform.CRM_RISK.fraud_blacklist_clean),
educate as 
    (select distinct edu.digitalLoanAccountId, edu.education_id, edu1.description
    from `prj-prod-dataplatform.dl_loans_db_raw.tdbk_loan_purpose` edu
    inner join (select id, description from dl_loans_db_raw.tdbk_loan_lov_mtb where module = 'Education') edu1 on edu.education_id = edu1.id
    ),
    educate2 as 
    (select *, row_number() over(partition by digitalLoanAccountId order by education_id desc) rnk from educate),
    educate3 as 
    (select * from educate2 where rnk = 1),
    education as 
    (
    select 
    lmt.customerId,
    educate3.digitalLoanAccountId,
    educate3.education_id,
    educate3.description Education_Type, 
    educate3.rnk educationrnk
    from educate3 
    inner join prj-prod-dataplatform.risk_credit_mis.loan_master_table lmt on lmt.digitalLoanAccountId = educate3.digitalLoanAccountId
    )
    ,
    digitalloanapplication as (select 
    digitalLoanAccountId, 
    monthlyInterestRate , 
    interest_rate_offer ,
    monthlyIncomeCheck, 
    from prj-prod-dataplatform.dl_loans_db_raw.tdbk_digital_loan_application ), 
    loantypeselection as 
    (SELECT 
    digitalLoanAccountId 
    FROM `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 
    where date(disbursementDateTime) >= '2023-07-01'
    and flagDisbursement = 1
    and kycStatus = 'SKYC'
    -- and customerId in (select customerId from Excl_evertransacted) ---Excluded customers who never transacted
    -- and loanAccountNumber in (select loanAccountNumber from Excl_qualified_FSPD30) --- Exclude customer who has not yet matured FSPD30
    -- and customerId not in (select distinct customerid from Excl_blacklisted_clean) --- Exclude customer who are blacklisted
    -- and customerId not in (select distinct customerId from `risk_credit_mis.loan_master_table` where loanPaidStatus in ('Written Off', 'Sold')) --- Exclude written off
    -- and customerId not in (Select PID from dap_ds_poweruser_playground.Defraudedandhacked_mu where PID is not null)
    ),
    Appreinstall as 
    (select cast(customer_id as numeric) customerId, count(customer_id)  from moengage_raw.events_hourly where event_code = 'REINSTALL'
        group by 1
        having count(customer_id) > 1)
    ,
    loan
    as
    (select 
    distinct
    lmt.digitalLoanAccountId ,
    lmt.customerId ,
    lmt.loanAccountNumber ,
    lmt.tsa_onboarding_time,
    lmt.startApplyDateTime,
    lmt.decision_date,
    lmt.termsAndConditionsSubmitDateTime,
    lmt.disbursementDateTime, 
    lmt.new_loan_type,
    lmt.gender,
    lmt.age,
    education.Education_Type,
    education.education_id,
    lmt.custLoanJourneyCity,
    lmt.custLoanJourneyProvince,
    lmt.custLoanJourneyZipCode,
    lmt.province,
    lmt.city,
    lmt.postalCode,
    lmt.incomeperSalaryScale,
    lmt.monthlyIncome,
    lmt.maritalStatus,
    lmt.purposeDescription,
    lmt.isfatca,
    lmt.docType,
    lmt.loanRequestAmount,
    lmt.loanRequestTenure,
    dlp.monthlyInterestRate,
    dlp.interest_rate_offer,
    dlp.monthlyIncomeCheck,
    lmt.kycStatus,
	case when lower(coalesce(lmt.osVersion, lmt.osversion_v2)) like '%ios%' then 'ios'
	     when lower(coalesce(lmt.osVersion, lmt.osversion_v2)) like '%andro%' then 'android'
		 else 'other' end Device_os_type,
    delqtab.obsFSPD30,
    delqtab.FSPD30,
	  case when lower(coalesce(lmt.osVersion, lmt.osversion_v2)) like '%ios%' then coalesce(cast(SUBSTRING(coalesce(lmt.osVersion, lmt.osversion_v2), 4, 2)  as numeric),0)
	     when lower(coalesce(lmt.osVersion, lmt.osversion_v2)) like '%andro%' then coalesce(cast(SUBSTRING(coalesce(lmt.osVersion, lmt.osversion_v2), 8, 2)  as numeric),0)
		 end Device_OSVersion,
    b.Network Network_provider,
    lmt.brand,
	case when eet.customerId is null then 1 else 0 end customer_never_transacted_flag,
	case when eqf.loanAccountNumber is not null then 1 else 0 end fspd30_mature_flag,
	case when ebc.customerId is not null then 1 else 0 end blacklisted_customer_flag,
	case when lmt.customerId in (select distinct customerId from `risk_credit_mis.loan_master_table` where loanPaidStatus in ('Written Off', 'Sold')) then 1 else 0 end writtenoff_flag,
	case when lmt.customerId in (Select PID from dap_ds_poweruser_playground.Defraudedandhacked_mu where PID is not null) then 1 else 0 end hackedordefrauded_flag,
  case when e.vas_flag = 'true' then 1 else 0 end as VasFlag,
  case when Appreinstall.customerId is not null then 1 else 0 end EverReinstallFlag,
  lmt.credolabScore
    from base 
    left join   `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = base.digitalLoanAccountId
    inner join loantypeselection lts on lts.digitalLoanAccountId = lmt.digitalLoanAccountId 
    inner join delqtab on delqtab.loanAccountNumber = lmt.loanAccountNumber
    Left join education on education.digitalLoanAccountId = lmt.digitalLoanAccountId
    left join digitalloanapplication dlp on dlp.digitalLoanAccountId = lmt.digitalLoanAccountId
    LEFT JOIN (select *, row_number() over(partition by Number_Prefix order by Network) rnk 
  from `prj-prod-dataplatform.manual_source_extracts.mobile_carrier_mapping`) b ON LEFT(RIGHT(lmt.mobileNo,LENGTH(lmt.mobileNo)-2),3) = CAST(b.number_prefix AS STRING)
	 left join Excl_evertransacted eet on eet.customerId = lmt.customerId
	 left join Excl_qualified_FSPD30 eqf on eqf.loanAccountNumber = lmt.loanAccountNumber
	 left join Excl_blacklisted_clean ebc on ebc.customerId = lmt.customerId
   left join prj-prod-dataplatform.dl_loans_db_raw.tdbk_digital_loan_application e
          on e.loanAccountNumber = lmt.loanAccountNumber
   left join Appreinstall on Appreinstall.customerId = lmt.customerId
    where b.rnk = 1
    )
    select * from loan ;


    """

In [15]:
df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID f6961aff-eadb-467e-bcc3-01ec0210f4f7 successfully executed: |          | 
Downloading: 100%|██████████|


In [16]:
df.shape 

(128889, 45)

# Events Before Apply

In [35]:
sq = """with eventbase as 
(select Customer_User_ID,Event_Time, Event_Name  from prj-prod-dataplatform.appsflyer_raw.in_app_events_report where Customer_User_ID is not null 
union all
select Customer_User_ID,Event_Time, Event_Name from prj-prod-dataplatform.appsflyer_raw.organic_in_app_events_report where Customer_User_ID is not null 
),
eventsbeforeapply as 
(select a.customerId, a.digitalLoanAccountId , a.startApplyDateTime 
, eb.Customer_User_ID, eb.Event_Time, eb.Event_Name
from `risk_credit_mis.loan_master_table` a
left join eventbase eb on eb.Customer_User_ID  = cast(a.customerId as string) and date(a.startApplyDateTime) > date(eb.Event_Time)
),
eventsbeforeapplybase as
(select digitalLoanAccountId, startApplyDateTime, count(distinct Event_Name) cntevents 
from eventsbeforeapply
group by 1,2
)
select * from eventsbeforeapplybase;"""

eventbfrapplydf = client.query(sq).to_dataframe(progress_bar_type='tqdm')
eventbfrapplydf

Job ID b97fe9af-6d8c-48d2-a07e-9536efd7e2ba successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,digitalLoanAccountId,startApplyDateTime,cntevents
0,b4aaca62-ff8f-4722-8232-478a1c3dfd9f,2024-06-15 16:26:18,57
1,229b6df3-688d-448b-ad1e-aac9a7e66c79,2022-12-13 11:56:36,110
2,045a6607-8c51-49c5-b1b5-db396ed7f588,2023-11-08 23:08:28,76
3,c7577e08-fb1a-4c13-86b3-992627767910,2024-05-26 15:16:13,34
4,b4d1074c-f7b8-4e5f-bd56-6cb3c5970d72,2022-12-24 01:03:04,31
...,...,...,...
1796427,42d60ef9-0e68-4f43-a8b2-bb2b7f4d1463,NaT,0
1796428,38bff4f5-f357-4c09-bbb7-83eed3eb81cc,2024-09-29 14:34:52,0
1796429,391abc27-bb64-40d1-8e7b-41dad63b67ad,2023-02-03 14:25:01,0
1796430,2c159f83-fbca-475c-b4c1-66d089ccc8a1,2022-12-14 23:45:16,0


# Saving the dataframes to csv files

In [36]:
dfhacked1.to_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\Demo&Telco Scorecard\Data\hackedconfirmedbyoperationsteam.csv", index = False)
dfaquisition.to_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\Demo&Telco Scorecard\Data\aquisitiondatareceivedfromIvanah.csv", index = False)
df.to_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\Demo&Telco Scorecard\Data\otherpart.csv", index = False)
eventbfrapplydf.to_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\Demo&Telco Scorecard\Data\eventsbeforeapplydate.csv", index = False)

In [37]:
finaldf = df.merge(dfaquisition, on='digitalLoanAccountId', how = 'left')
finaldf = finaldf.merge(eventbfrapplydf[['digitalLoanAccountId', 'cntevents']], on='digitalLoanAccountId', how = 'left')
finaldf.shape

(128889, 50)

In [38]:
finaldf.sample(5)

,digitalLoanAccountId,customerId_x,loanAccountNumber,tsa_onboarding_time,startApplyDateTime,decision_date,termsAndConditionsSubmitDateTime,disbursementDateTime,new_loan_type,gender,age,Education_Type,education_id,custLoanJourneyCity,custLoanJourneyProvince,custLoanJourneyZipCode,province,city,postalCode,incomeperSalaryScale,monthlyIncome,maritalStatus,purposeDescription,isfatca,docType,loanRequestAmount,loanRequestTenure,monthlyInterestRate,interest_rate_offer,monthlyIncomeCheck,kycStatus,Device_os_type,obsFSPD30,FSPD30,Device_OSVersion,Network_provider,brand,customer_never_transacted_flag,fspd30_mature_flag,blacklisted_customer_flag,writtenoff_flag,hackedordefrauded_flag,VasFlag,EverReinstallFlag,credolabScore,customerId_y,Appsflyer_id,Source,flag_organic_inorganic,cntevents
4706,fc94d0ff-9606-4956-8c78-92c06f24dc1e,2481751,60824817510016,2024-04-25 18:20:44,2024-04-25 18:24:16,2024-04-25 18:30:31,2024-04-25 18:30:26,2024-04-25 18:34:47,SIL-Instore,M,64,High School Graduate,11,ANGELES CITY,PAMPANGA,2009,Pampanga,ANGELES CITY,2009,45000,45000,Married,"Washing machine, dryer or both",0,Philippines - Id Card (2020),9091.0,8,5.040000,None,false,SKYC,android,1,0,13.000000000,Globe/TM,vivo,0,1,0,0,0,0,0,357,<NA>,NaN,NaN,NaN,0
5091,366c8dc4-06a2-4640-abba-9627002273f4,2511297,60825112970016,2024-05-12 14:20:44,2024-05-12 14:29:51,2024-05-12 14:38:22,2024-05-12 14:37:19,2024-05-12 14:44:19,SIL-Instore,M,22,College Undergraduate,13,BALAYAN,BATANGAS,4213,Batangas,BALAYAN,4213,15842.00,10000,Single,Refrigerators,0,Philippines - Id Card (2020),14798.0,10,5.080000,None,true,SKYC,android,1,0,13.000000000,Globe/TM,realme,0,1,0,0,0,0,0,433,<NA>,NaN,NaN,NaN,0
1762,2aa602b3-f328-4756-95d5-0353beb752de,2562029,60825620290017,2024-06-08 21:49:00,2024-06-13 06:12:29,2024-06-14 08:46:33,2024-06-13 06:19:28,2024-06-14 18:38:08,Quick,F,30,College Graduate,14,DAVAO CITY,DAVAO DEL SUR,8000,Davao del Sur,DAVAO CITY,8000,31757.00,50000,Single,Living Expenses,0,Philippines - Driving License,20000.0,12,7.916667,None,true,SKYC,ios,1,0,17.000000000,Globe/TM,Apple,0,1,0,0,0,0,0,None,2562029,1717853944861-0871534,Organic,Organic,107
63664,8b427ba5-32cb-499f-a9d6-1370716f0da5,2664956,60826649560011,2024-07-16 15:53:39,2024-07-16 15:56:48,2024-07-16 16:04:41,2024-07-16 16:04:26,2024-07-16 16:25:19,SIL-Instore,M,29,College Graduate,14,SAN MATEO,RIZAL,1850,Rizal,SAN MATEO,1850,20065.00,18000,Married,Bed frames or Cribs,0,Philippines - Id Card (2020),7099.0,6,6.520000,None,true,SKYC,android,0,0,12.000000000,Globe/TM,Infinix,0,0,0,0,0,0,0,422.0,<NA>,NaN,NaN,NaN,0
63013,c27a1008-a9e5-4223-8232-686172f5f303,2437355,60824373550017,2024-03-22 14:57:53,2024-03-22 15:02:44,2024-03-22 15:04:24,2024-03-22 15:04:20,2024-03-22 15:08:12,SIL-Instore,F,54,High School Graduate,11,LOS BANOS,LAGUNA,4030,Laguna,LOS BANOS,4030,103620.00,18000,Married,Televisions,0,Philippines - Id Card (2020),5999.0,6,4.500000,None,true,SKYC,android,1,0,13.000000000,TNT,vivo,0,1,0,0,0,0,0,436,<NA>,NaN,NaN,NaN,0


In [39]:
finaldf.columns

Index(['digitalLoanAccountId', 'customerId_x', 'loanAccountNumber',
       'tsa_onboarding_time', 'startApplyDateTime', 'decision_date',
       'termsAndConditionsSubmitDateTime', 'disbursementDateTime',
       'new_loan_type', 'gender', 'age', 'Education_Type', 'education_id',
       'custLoanJourneyCity', 'custLoanJourneyProvince',
       'custLoanJourneyZipCode', 'province', 'city', 'postalCode',
       'incomeperSalaryScale', 'monthlyIncome', 'maritalStatus',
       'purposeDescription', 'isfatca', 'docType', 'loanRequestAmount',
       'loanRequestTenure', 'monthlyInterestRate', 'interest_rate_offer',
       'monthlyIncomeCheck', 'kycStatus', 'Device_os_type', 'obsFSPD30',
       'FSPD30', 'Device_OSVersion', 'Network_provider', 'brand',
       'customer_never_transacted_flag', 'fspd30_mature_flag',
       'blacklisted_customer_flag', 'writtenoff_flag',
       'hackedordefrauded_flag', 'VasFlag', 'EverReinstallFlag',
       'credolabScore', 'customerId_y', 'Appsflyer_id', 'Source'

In [40]:
finaldf.rename(columns={'customerId_x':'customerId'}, inplace = True)

In [41]:
finaldf = finaldf[['digitalLoanAccountId', 'customerId', 'loanAccountNumber',
       'tsa_onboarding_time', 'startApplyDateTime', 'decision_date',
       'termsAndConditionsSubmitDateTime', 'disbursementDateTime','new_loan_type', 'gender', 'age', 'Education_Type',
       'custLoanJourneyCity', 'custLoanJourneyProvince',
       'custLoanJourneyZipCode', 'province', 'city', 'postalCode',
       'incomeperSalaryScale', 'monthlyIncome', 'maritalStatus',
       'purposeDescription', 'isfatca', 'docType', 'loanRequestAmount',
       'loanRequestTenure', 'monthlyInterestRate', 'interest_rate_offer',
       'monthlyIncomeCheck', 'kycStatus', 'Device_os_type','brand', 'obsFSPD30',
       'FSPD30', 'Device_OSVersion', 'Network_provider',
       'customer_never_transacted_flag','blacklisted_customer_flag', 'writtenoff_flag',
       'hackedordefrauded_flag', 'VasFlag', 'EverReinstallFlag',
       'Appsflyer_id', 'Source', 'flag_organic_inorganic', 'cntevents'  ]].copy()

In [42]:
finaldf.sample(5)

,digitalLoanAccountId,customerId,loanAccountNumber,tsa_onboarding_time,startApplyDateTime,decision_date,termsAndConditionsSubmitDateTime,disbursementDateTime,new_loan_type,gender,age,Education_Type,custLoanJourneyCity,custLoanJourneyProvince,custLoanJourneyZipCode,province,city,postalCode,incomeperSalaryScale,monthlyIncome,maritalStatus,purposeDescription,isfatca,docType,loanRequestAmount,loanRequestTenure,monthlyInterestRate,interest_rate_offer,monthlyIncomeCheck,kycStatus,Device_os_type,brand,obsFSPD30,FSPD30,Device_OSVersion,Network_provider,customer_never_transacted_flag,blacklisted_customer_flag,writtenoff_flag,hackedordefrauded_flag,VasFlag,EverReinstallFlag,Appsflyer_id,Source,flag_organic_inorganic,cntevents
79990,9c87b265-5868-471b-ad41-88ebc043641b,2476925,60824769250015,2024-04-22 17:09:12,2024-04-22 18:27:45,2024-04-22 18:38:18,2024-04-22 18:38:13,2024-04-23 16:14:57,SIL-Instore,F,34,College Graduate,QUEZON CITY,METRO MANILA,1105,METRO MANILA,QUEZON CITY,1105,23238.00,23000,Single,Mobile Phones,0,Philippines - UMID Card (2016),26990.0,12,4.290000,None,true,SKYC,ios,Apple,1,0,17.000000000,Globe/TM,0,0,0,0,0,0,NaN,NaN,NaN,0
38337,d899e25a-ad73-4140-b78d-e62b5efbf805,2232969,60822329690019,2023-09-14 11:47:10,2023-09-14 11:58:22,2023-09-14 12:00:13,2023-09-14 12:00:08,2023-09-14 12:17:15,SIL-Instore,F,20,None,IMUS,CAVITE,4103,Cavite,IMUS,4103,34460.00,40000,Single,"Televisions,Refrigerators",0,Philippines - ePassport (2016),20194.0,6,4.500000,None,true,SKYC,android,vivo,1,0,12.000000000,Globe/TM,0,0,0,0,0,0,NaN,NaN,NaN,0
8807,23d32202-80de-4bcd-984a-b72ae82e10a3,2801128,60828011280013,2024-08-28 14:35:41,2024-08-28 14:41:29,2024-08-28 14:46:06,2024-08-28 14:45:45,2024-08-28 14:47:48,SIL-Instore,F,40,Technical/Vocational Graduate,CASTILLEJOS,ZAMBALES,2208,Zambales,CASTILLEJOS,2208,39317.00,20000,Widow / Widower,"Mobile Phones,Accessories or Peripherals",0,Philippines - UMID Card (2016),7787.0,10,5.470000,None,true,SKYC,android,TECNO,0,0,11.000000000,TNT,1,0,0,0,1,0,NaN,NaN,NaN,0
6602,01060bb4-9529-4bdf-bac6-6544b1433ad3,2471892,60824718920013,2024-04-19 14:54:55,2024-08-31 12:50:17,2024-08-31 12:57:18,2024-08-31 12:54:23,2024-08-31 13:46:36,Quick,M,29,College Graduate,LUCBAN,QUEZON,4328,Quezon,LUCBAN,4328,67304.00,50000,Single,Medical or Other Emergency/Health Care,0,Philippines - ePassport,10000.0,9,7.916667,None,true,SKYC,android,samsung,0,0,13.000000000,Globe/TM,0,0,0,0,1,0,1713509321120-4977873843052323751,af_Banner,InOrganic,70
59125,601319cc-72a9-47f5-bb68-2156aa6fd037,2534432,60825344320017,2024-05-26 14:23:25,2024-05-26 14:37:02,2024-05-26 14:42:33,2024-05-26 14:42:28,2024-05-26 14:50:31,SIL-Instore,F,31,High School Graduate,ANTIPOLO,RIZAL,1870,Rizal,ANTIPOLO,1870,15000,15000,Married,"Dining set, Chairs or Tables,Bed frames or Cribs",0,Philippines - Id Card (2020),6792.0,5,4.850000,None,false,SKYC,android,HONOR,1,0,13.000000000,TNT,0,0,0,0,0,0,NaN,NaN,NaN,18


In [43]:
finaldf.to_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\Demo&Telco Scorecard\Data\demographic_data_20241009.csv", index = False)

In [3]:
readdf = pd.read_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\Demo&Telco Scorecard\Data\demographic_data_20241009.csv")

In [4]:
readdf.shape

(128889, 46)

In [7]:
readdf.dtypes

digitalLoanAccountId                 object
customerId                            int64
loanAccountNumber                     int64
tsa_onboarding_time                  object
startApplyDateTime                   object
decision_date                        object
termsAndConditionsSubmitDateTime     object
disbursementDateTime                 object
new_loan_type                        object
gender                               object
age                                   int64
Education_Type                       object
custLoanJourneyCity                  object
custLoanJourneyProvince              object
custLoanJourneyZipCode              float64
province                             object
city                                 object
postalCode                            int64
incomeperSalaryScale                float64
monthlyIncome                         int64
maritalStatus                        object
purposeDescription                   object
isfatca                         

In [9]:
pd.set_option("Display.max_columns", None)
readdf.head()

,digitalLoanAccountId,customerId,loanAccountNumber,tsa_onboarding_time,startApplyDateTime,decision_date,termsAndConditionsSubmitDateTime,disbursementDateTime,new_loan_type,gender,age,Education_Type,custLoanJourneyCity,custLoanJourneyProvince,custLoanJourneyZipCode,province,city,postalCode,incomeperSalaryScale,monthlyIncome,maritalStatus,purposeDescription,isfatca,docType,loanRequestAmount,loanRequestTenure,monthlyInterestRate,interest_rate_offer,monthlyIncomeCheck,kycStatus,Device_os_type,brand,obsFSPD30,FSPD30,Device_OSVersion,Network_provider,customer_never_transacted_flag,blacklisted_customer_flag,writtenoff_flag,hackedordefrauded_flag,VasFlag,EverReinstallFlag,Appsflyer_id,Source,flag_organic_inorganic,cntevents
0,1a99c362-19b4-44b4-8ebb-0ac27d2768a3,2302133,60823021330014,2023-11-12 17:00:07,2023-11-12 17:05:56,2023-11-12 17:13:44,2023-11-12 17:13:40,2023-11-12 17:24:43,SIL-Instore,F,32,College Graduate,PILILLA,RIZAL,1910.0,Rizal,PILILLA,1910,19202.0,20000,Single,"Sala set, Couch, Sofa set",0,Philippines - Driving License (2022),10000.0,3,4.500000,NaN,True,SKYC,android,TECNO,1,0,10.0,TNT,0,0,0,0,0,0,NaN,NaN,NaN,0
1,9c4fd55f-0b3a-4ac3-b559-fe63260378ca,2307189,60823071890011,2023-11-17 18:01:56,2023-11-17 18:03:28,2023-11-17 18:07:34,2023-11-17 18:07:30,2023-11-17 18:12:10,SIL-Instore,M,31,College Graduate,TALISAY,BATANGAS,4220.0,Batangas,TALISAY,4220,44734.0,30000,Married,Refrigerators,0,Philippines - Professional Driving License (2017),12295.0,12,4.500000,NaN,True,SKYC,android,realme,1,0,13.0,Globe/TM,0,0,0,0,1,0,NaN,NaN,NaN,0
2,59979aa9-04a3-484a-bc5d-fb96d99972f2,2513333,60825133330011,2024-05-13 16:49:29,2024-06-26 11:08:16,2024-06-26 15:05:19,2024-06-26 11:21:16,2024-06-26 17:13:18,Quick,F,44,College Graduate,KAWIT,CAVITE,4104.0,Cavite,KAWIT,4104,27183.0,29100,Single,Education,0,Philippines - UMID Card (2021),5000.0,6,7.916667,NaN,True,SKYC,android,samsung,1,1,11.0,Smart,0,0,0,0,1,0,1715589519410-15553018338392167,Organic,Organic,19
3,bc5a74b5-e8dd-459f-a18f-067f7832c532,2205748,60822057480015,2023-08-24 19:05:47,2023-08-24 19:20:41,2023-08-24 19:28:27,2023-08-24 19:28:23,2023-08-24 19:31:51,SIL-Instore,M,37,NaN,GLORIA,ORIENTAL MINDORO,5209.0,Mindoro Oriental,GLORIA,5209,19202.0,22000,Single,"Dining set, Chairs or Tables",0,Philippines - UMID Card (2016),7396.0,9,4.500000,NaN,True,SKYC,android,samsung,1,0,13.0,Globe/TM,0,0,0,0,0,0,NaN,NaN,NaN,0
4,7fe8f81f-7ca3-41e3-a312-747368fd0ef7,2809734,60828097340011,2024-08-31 10:06:58,2024-08-31 10:11:38,2024-08-31 10:17:22,2024-08-31 10:17:08,2024-08-31 11:04:30,SIL-Instore,F,39,High School Graduate,SARIAYA,QUEZON,4322.0,Quezon,SARIAYA,4322,30000.0,30000,Married,Televisions,0,Philippines - Id Card (2020),7999.0,6,6.520000,NaN,NaN,SKYC,android,Infinix,0,0,12.0,DITO,1,0,0,0,1,0,NaN,NaN,NaN,0
